In [2]:
try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen

from datetime import date, timedelta, datetime
import pymongo
import json
import getSchedules

In [3]:
#Open a connection to the obsLog mongoDB database
conn = pymongo.MongoClient('observinglogs,observinglogs2,observinglogs3',replicaSet='KEOLA')
db = conn.obsLog

In [9]:
errors = []
d = datetime(2018,12,31)
d

datetime.datetime(2018, 12, 31, 0, 0)

In [17]:
def fromWeb( inDate ):
    """ Pulls schedules for a given date from the website and parses them into a dictionary """
    url = "http://www/observing/schedule/ws/telsched.php?date=" + inDate.strftime("%Y-%m-%d") + "&tel="

    schedules = {}
    for tel in [1,2]:
        schedules[ tel ] = {}
        print(url)
        response = urlopen( url + str(tel) )
        #result = response.read()
        #print(response)
        for line_number,line in enumerate(response):
        #sys.exit(0)
        #for s in result.split("\n"):
        #for s in result:
            s=line.decode('utf-8')
            #print (s)
            if "=" in s:
                kv = s.split("=")
                schedules[ tel ][ kv[0] ] = kv[1]
    return schedules
def parseTwilight( sched ):
    """ Parses a schedule's twilight entry into a dictionary """
    # Parse schedule's Twilight dictionary for storage in logs
    # ( This ugly looking one liner essentially just parses
    #   what is the serialization of a dictionary into a string
    #   back into the actual dictionary it represents )
    return dict( [[ y.strip("'") for y in x.split(":", 1)] for x in sched["Twilight"].strip("{}").split(",") ]  )

def findInstrEntry( dbInstruments, instrStr ):
    """ Find the matching instrument db entry for a given instrument string """
    for instrument in dbInstruments:
        if instrument["name"] in instrStr:
            return instrument
    return instrStr

def genLogs( d, db, errors ):
    """ Parses out database log entries from the schedule for a given date """

    # An empty list to append output logs
    outLogs = []

    # Get a list of all the entries in the "instruments" collection
    dbInstruments = [x for x in db.instruments.find()]

    # List of months to be used for really lame lower casing method
    mons = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

    # Pull logs for this date
    for sched in fromWeb( d ).values():
        #print("******************** SCHEDULE ENTRY ******************")
        #print("Sched: "+str(sched))

        # Implentation of Dr. Rizzi's finite state machine algorithm for
        # parsing schedule entries.  The result we want is an array
        # which groups together any instrument account entries that
        # are sperated with a +, each grouping being an array of
        # dictionaries that have the form
        # { "instrEntry": ( database entry for the matching instrument,
        #                   or original instrument string if entry not found )
        #   "accountID" : ( The string of the parsed account ID or "" if none found )
        # }

        instrAcc = sched["InstrAcc"]
        obsList = [[]]
        inParens = False
        foundParens = False
        curInstrument = False
        splitPos = 0
        for i, ch in enumerate( instrAcc ):
            if ch == "(":
                foundParens = True
                inParens = True
                curInstrument = findInstrEntry( dbInstruments, instrAcc[ splitPos : i ] )
                splitPos = i+1
            elif ch == ")":
                inParens = False
                obsList[-1].append( { "instrEntry": curInstrument, "accountID":  instrAcc[ splitPos : i ] } )
            elif ch == "/" and inParens:
                obsList[-1].append( { "instrEntry": curInstrument, "accountID":  instrAcc[ splitPos : i ] } )
                splitPos = i+1
                obsList.append([])
            elif ch == "/" or ch == "+" or i == ( len(instrAcc)-1 ):
                if i == ( len(instrAcc)-1 ):
                    i+=1
                if not foundParens:
                    curInstrument = findInstrEntry( dbInstruments, instrAcc[ splitPos : i ] )
                    obsList[-1].append( { "instrEntry": curInstrument, "accountID": "" } )
                foundParens = False
                splitPos = i+1
                curInstrument = False
                if ch == "/":
                    obsList.append([])

        # Now that we have them split into separate entries and grouped by observers
        # we go through and try to generate the dataDirs list for each log

        # We will need to keep track of an account+id combos that appear
        # more than once so that we can append "_B", "_C" etc to the
        # date portion of their directory names
        splitTally = {}

        for obsGroup in obsList:
            #print("******************** OBSGROUP ENTRY ******************")
            #print(str(obsList))
            for obs in obsGroup:
                #print (type(obs['instrEntry']))
                if type(obs["instrEntry"]) is not dict:
                    errors.append("Omit: No instrument found matching " + obs["instrEntry"] )
                    obs["instrEntry"] = False
                elif obs["accountID"] == "":
                    try:
                        errors.append("Omit: No account ID found for "+ obs["instrEntry"]["name"]+" log." )
                    except:
                        errors.append("Omit: No account ID found for "+ str(obs["instrEntry"])+" log." )
                    obs["instrEntry"] = False
                else:
                    instr = obs["instrEntry"]
                    if (instr["dirName"]+obs["accountID"]) not in splitTally:
                        # The first of the duplicate instrument+id combos
                        # doesn't need anything appended after the date
                        obs["dateSuffix"] = ""
                        # Create an initial tally entry that will be incremented
                        # every new duplication of this instrument+id combo
                        splitTally[ instr["dirName"]+obs["accountID"] ] = "_A"
                    else:
                        # Pull the relevant tally
                        tal = splitTally[ instr["dirName"]+obs["accountID"] ]
                        # Increment it "_A" -> "_B", etc.
                        tal = tal[:-1] + chr( ord( tal[-1] ) + 1 )
                        splitTally [instr["dirName"]+obs["accountID"]] = tal

                        # Store it for later appending
                        obs["dateSuffix"] = tal

                    # Increment day by one, and store it in the observation
                    obs["date"] = (d+timedelta(days=1))

                    # Create dataDirs specific to this log entry by supplying
                    # this obs entry to the format string stored for each protoDir
                    obs["dataDirs"] = []
                    for pd in instr["protoDirs"]:
                        dir =  pd.format( **obs )

                        # Really lame way of lowercasing the months because I can't
                        # figure out any way to simply lower case within format spec
                        for m in mons:
                            if m in dir:
                                dir = dir.replace(m, m.lower() )

                        obs["dataDirs"].append( dir )


        # Now attempt to actually create the logs, appending details from the schedule if possible

        projSplit = sched["ProjCode"].split("/")
        obsSplit = sched["Observers"].split("/")
        piSplit = sched["Principal"].split("/")

        for i in range(len( obsList )) :
            for obs in [x for x in obsList[i] if x["instrEntry"]]:
                protoLog = {"instrument": obs["instrEntry"]["name"],
                    "project": "",
                    "observers": "",
                    "pi": "",
                    "sa": sched["SA"].strip(),
                    "oa": sched["OA"].strip(),
                    "utcDate": datetime.utcnow(),
                    "activeDirs": [],
                    "dataDirs": obs["dataDirs"] }
                # observers
                if i < len( obsSplit ) and obsSplit[i].strip()!="":
                    protoLog["observers"] = obsSplit[ i ].strip()
                else:
                    errors.append("Warning: Failed to find observers for "+obs["instrEntry"]["name"]+" log" )
                # projects
                if i < len( projSplit ) and projSplit[i].strip()!="":
                    protoLog["project"] = projSplit[ i ].strip()
                else:
                    errors.append("Warning: Failed to find project code for "+obs["instrEntry"]["name"]+" log")
                # PIs
                if i < len( piSplit ) and piSplit[i].strip()!="":
                    protoLog["pi"] = piSplit[ i ].strip()
                else:
                    errors.append("Warning: Failed to find PI information for "+obs["instrEntry"]["name"]+" log")

                outLogs.append( protoLog )
        
    # Return the log entries generated
    return outLogs

In [3]:
def getSchedule( inDate, telescope ):
    """ Pulls schedules for a given date from the website and parses them into a dictionary """
    url = "https://www.keck.hawaii.edu/software/db_api/telSchedule.php?cmd=getSchedule&date=" + inDate.strftime("%Y-%m-%d")

    url = url + "&telnr=%d" % telescope
    response = urlopen( url )
    print(url)
    result = response.read().decode('utf-8')
    result = json.loads(result)
    return result

def getNightStaff( inDate ):
    """ Pulls schedules for a given date from the website and parses them into a dictionary """
    url = "https://www.keck.hawaii.edu/software/db_api/telSchedule.php?cmd=getNightStaff&date=" + inDate.strftime("%Y-%m-%d")

    schedules = []
    
    response = urlopen( url )
    result = response.read().decode('utf-8')
    
    return json.loads(result)

In [20]:
oldlogs = genLogs(d, db, errors)

http://www/observing/schedule/ws/telsched.php?date=2018-12-21&tel=
http://www/observing/schedule/ws/telsched.php?date=2018-12-21&tel=


In [11]:
newlogs = getSchedules.genLogs(d, db, errors)
newlogs

https://www.keck.hawaii.edu/software/db_api/telSchedule.php?cmd=getSchedule&date=2018-12-31&telnr=1
Looking for database entry for instrument: <LRIS-ADC>
Looking for database entry for instrument: <LRIS-ADC>
https://www.keck.hawaii.edu/software/db_api/telSchedule.php?cmd=getSchedule&date=2018-12-31&telnr=2
Looking for database entry for instrument: <KCWI>
Looking for database entry for instrument: <KCWI>


[{'project': 'N044',
  'observers': 'Masters,Cohen,Hernitschek,Davidzon',
  'pi': 'Masters',
  'oa': 'Joel Aycock',
  'utcDate': datetime.datetime(2019, 1, 1, 23, 4, 43, 23504),
  'instrument': 'LRIS',
  'dataDirs': ['/s/sdata243/lris9/2019jan01'],
  'activeDirs': []},
 {'project': 'N188',
  'observers': 'Prichard,Cooke,Mestric,Webb',
  'pi': 'Rafelski',
  'oa': 'Joel Aycock',
  'utcDate': datetime.datetime(2019, 1, 1, 23, 4, 43, 24445),
  'instrument': 'LRIS',
  'dataDirs': ['/s/sdata243/lris9/2019jan01_B'],
  'activeDirs': []},
 {'project': 'C308',
  'observers': 'Hamden',
  'pi': 'Hamden',
  'oa': 'Terry Stickel',
  'sa': 'Luca Rizzi',
  'utcDate': datetime.datetime(2019, 1, 1, 23, 4, 43, 66498),
  'instrument': 'KCWI',
  'dataDirs': ['/s/sdata1400/kcwi7/2019jan01'],
  'activeDirs': []},
 {'project': 'N206',
  'observers': 'Rupke',
  'pi': 'Rupke',
  'oa': 'Terry Stickel',
  'sa': 'Luca Rizzi',
  'utcDate': datetime.datetime(2019, 1, 1, 23, 4, 43, 67509),
  'instrument': 'KCWI',
  '

In [13]:
night_staff = getSchedules.getNightStaff(d)
night_staff

[{'Date': '2018-12-31',
  'TelNr': '0',
  'Alias': 'echock',
  'Type': 'swoc',
  'LastName': 'Chock',
  'FirstName': 'Liz'},
 {'Date': '2018-12-31',
  'TelNr': '1',
  'Alias': 'jaycock',
  'Type': 'oa',
  'LastName': 'Aycock',
  'FirstName': 'Joel'},
 {'Date': '2018-12-31',
  'TelNr': '1',
  'Alias': 'arettura',
  'Type': 'saoc',
  'LastName': 'Rettura',
  'FirstName': 'Alessandro'},
 {'Date': '2018-12-31',
  'TelNr': '2',
  'Alias': 'julierk',
  'Type': 'oa',
  'LastName': 'Renaud-Kim',
  'FirstName': 'Julie'},
 {'Date': '2018-12-31',
  'TelNr': '2',
  'Alias': 'tstickel',
  'Type': 'oar',
  'LastName': 'Stickel',
  'FirstName': 'Terry'},
 {'Date': '2018-12-31',
  'TelNr': '2',
  'Alias': 'lrizzi',
  'Type': 'sa',
  'LastName': 'Rizzi',
  'FirstName': 'Luca'}]

In [7]:
newlogs

[{'project': 'N044',
  'observers': 'Masters,Cohen,Hernitschek,Stanford',
  'pi': 'Masters',
  'oa': 'Joel Aycock',
  'utcDate': datetime.datetime(2019, 1, 1, 23, 3, 16, 642970),
  'instrument': 'LRIS',
  'dataDirs': ['/s/sdata243/lris9/2019jan02'],
  'activeDirs': []},
 {'project': 'U090',
  'observers': 'Coil,Burchett,Vaught,Rubin',
  'pi': 'Fuller',
  'oa': 'Terry Stickel',
  'sa': 'Luca Rizzi',
  'utcDate': datetime.datetime(2019, 1, 1, 23, 3, 16, 716763),
  'instrument': 'KCWI',
  'dataDirs': ['/s/sdata1400/kcwi8/2019jan02'],
  'activeDirs': []}]

In [9]:
newlogs[1]['observers']

'Fan'

In [12]:
schedule

{'Date': '2018-12-23\n',
 'TelNr': '1\n',
 'Observers': 'L. Weiss; (IfA) / L. Weiss; (IfA) / \n',
 'Location': 'IfA/HQ\n',
 'InstrAcc': 'HIRESr(1/1)/PCS\n',
 'Instrument': 'HIRESr/HIRESr/PCS\n',
 'Institution': 'NASA/KECK\n',
 'Principal': 'Crossfield/Crossfield/Engineering\n',
 'ProjCode': 'N209/N015/E005\n',
 'Comment': 'split ',
 'Laser': '\n',
 'SA': 'Josh Walawender\n',
 'OA': 'John\n',
 'NA': 'Sniffen\n',
 'SWOC': 'Allan Honey\n',
 'Dark': '13\n',
 'DOW': 'Sunday\n',
 'EE': 'Chan\n',
 'Twilight': "{udate:'2018-12-24',dusk_12deg:'04:41:00',dusk_18deg:'05:08:00',dawn_18deg:'15:35:00',dawn_12deg:'16:02:00',dark:'13',sunset:'04:00:00',sunrise:'16:43:00',moonRADEC:'0856 18'}\n"}

In [13]:
parseTwilight(schedule)

{'udate': '2018-12-24',
 'dusk_12deg': '04:41:00',
 'dusk_18deg': '05:08:00',
 'dawn_18deg': '15:35:00',
 'dawn_12deg': '16:02:00',
 'dark': '13',
 'sunset': '04:00:00',
 'sunrise': '16:43:00',
 'moonRADEC': "0856 18'}\n"}

In [14]:
def getTwilight(inDate):
    """ Pulls twilight info for a given date from the website and parses them into a dictionary """
    url = "https://www.keck.hawaii.edu/software/db_api/metrics_api.php?date=" + inDate.strftime("%Y-%m-%d")

    response = urlopen( url )
    print(url)
    result = response.read().decode('utf-8')
    result = json.loads(result)
    return result


In [15]:
getTwilight(d)

https://www.keck.hawaii.edu/software/db_api/metrics_api.php?date=2018-12-23


{'udate': '2018-12-23',
 'dusk_12deg': '04:40:00',
 'dusk_18deg': '05:07:00',
 'dawn_18deg': '15:35:00',
 'dawn_12deg': '16:02:00',
 'dark': '3',
 'sunset': '04:00:00',
 'sunrise': '16:42:00',
 'moonRADEC': '0752 20'}

In [10]:
employee = getNightStaff(d)

In [11]:
employee

[{'Date': '2018-12-27',
  'TelNr': '0',
  'Alias': 'jlapinta',
  'Type': 'nah2',
  'LastName': 'La Pinta',
  'FirstName': 'Joshua'},
 {'Date': '2018-12-27',
  'TelNr': '0',
  'Alias': 'tconnors',
  'Type': 'nah2',
  'LastName': 'Connors',
  'FirstName': 'Tony'},
 {'Date': '2018-12-27',
  'TelNr': '0',
  'Alias': 'ahoney',
  'Type': 'swoc',
  'LastName': 'Honey',
  'FirstName': 'Allan'},
 {'Date': '2018-12-27',
  'TelNr': '1',
  'Alias': 'ahatakeyama',
  'Type': 'oa',
  'LastName': 'Hatakeyama',
  'FirstName': 'Alan'},
 {'Date': '2018-12-27',
  'TelNr': '1',
  'Alias': 'arostopchina',
  'Type': 'oao',
  'LastName': 'Rostopchina',
  'FirstName': 'Arina'},
 {'Date': '2018-12-27',
  'TelNr': '1',
  'Alias': 'syeh',
  'Type': 'sa',
  'LastName': 'Yeh',
  'FirstName': 'Sherry'},
 {'Date': '2018-12-27',
  'TelNr': '2',
  'Alias': 'tstickel',
  'Type': 'oa',
  'LastName': 'Stickel',
  'FirstName': 'Terry'},
 {'Date': '2018-12-27',
  'TelNr': '2',
  'Alias': 'pgomez',
  'Type': 'sa',
  'LastNam

In [30]:
for staff in filter(lambda nightstaff: (('oa' in nightstaff['Type']) and (nightstaff['TelNr'] == "2")),employee):
    print(staff)

{'Date': '2018-12-21', 'TelNr': '2', 'Alias': 'cwilburn', 'Type': 'oar', 'LastName': 'Wilburn', 'FirstName': 'Cynthia'}
